In [ ]:
#default_exp models

In [ ]:
#export
from timeseries_fastai.core import *
import torch
import torch.nn as nn
from fastcore.all import *
from fastai.basics import *
from fastai.torch_core import *
from fastai.layers import *
from fastai.vision import *

# Timeseries Models
> A list of timeseries neural architectures

In [ ]:
ts = torch.rand(2, 1, 20) # batch of 16 items with 1 channels and lenght 20

In [ ]:
#export
act_fn = nn.ReLU(inplace=True)

Same as `AdaptiveConcatPool2d` but on 1D

In [ ]:
#export
class AdaptiveConcatPool1d(nn.Module):
    "Layer that concats `AdaptiveAvgPool1d` and `AdaptiveMaxPool1d`"
    def __init__(self, size=None):
        super().__init__()
        self.size = size or 1
        self.ap = nn.AdaptiveAvgPool1d(self.size)
        self.mp = nn.AdaptiveMaxPool1d(self.size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

## Multi Layer Perceptron (Linear layered model)
A simple model builder to create a bunch of `BatchNorm1d`, `Dropout` and `Linear` layers, with `act_fn` activations.

In [ ]:
#export
def create_mlp(ni, nout, linear_sizes=[500, 500, 500]):
    layers = []
    sizes = zip([ni]+linear_sizes, linear_sizes+[nout])
    for n1, n2 in sizes:
            layers += LinBnDrop(n1, n2, p=0.2, act=act_fn if n2!=nout else None)
    return nn.Sequential(Flatten(),
                         *layers)

In [ ]:
mlp = create_mlp(1*20, 37)
mlp

Sequential(
  (0): Flatten(full=False)
  (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=20, out_features=500, bias=False)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.2, inplace=False)
  (7): Linear(in_features=500, out_features=500, bias=False)
  (8): ReLU(inplace=True)
  (9): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.2, inplace=False)
  (11): Linear(in_features=500, out_features=500, bias=False)
  (12): ReLU(inplace=True)
  (13): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): Dropout(p=0.2, inplace=False)
  (15): Linear(in_features=500, out_features=37, bias=False)
)

In [ ]:
mlp(ts).shape

torch.Size([2, 37])

## Fully Convolutional Network (FCN).
> A bunch of convolutions stacked together.

In [ ]:
#export
def create_fcn(ni, nout, ks=9, conv_sizes=[128, 256, 128], stride=1):
    layers = []
    sizes = zip([ni]+conv_sizes, conv_sizes)
    for n1, n2 in sizes:
            layers += [ConvLayer(n1, n2, ks=ks, ndim=1, stride=stride)]
    return nn.Sequential(*layers, 
                         AdaptiveConcatPool1d(),
                         Flatten(),
                         *LinBnDrop(2*n2, nout)
                         )

In [ ]:
fcn = create_fcn(1, 37)
fcn

Sequential(
  (0): ConvLayer(
    (0): Conv1d(1, 128, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv1d(128, 256, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv1d(256, 128, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): AdaptiveConcatPool1d(
    (ap): AdaptiveAvgPool1d(output_size=1)
    (mp): AdaptiveMaxPool1d(output_size=1)
  )
  (4): Flatten(full=False)
  (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=256, out_features=37, bias=False)
)

In [ ]:
fcn(ts).shape

torch.Size([2, 37])

## Resnet

In [ ]:
#export
def res_block_1d(nf, ks=[5,3]):
    "Resnet block as described in the paper."
    return SequentialEx(ConvLayer(nf, nf, ks=ks[0], ndim=1, ),
                        ConvLayer(nf, nf, ks=ks[1], ndim=1, act_cls=None),
                        MergeLayer())

In [ ]:
resb = res_block_1d(16)
resb

SequentialEx(
  (layers): ModuleList(
    (0): ConvLayer(
      (0): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): ConvLayer(
      (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): MergeLayer()
  )
)

In [ ]:
resb(torch.rand(2, 16, 100)).shape

torch.Size([2, 16, 100])

In [ ]:
#export
def create_resnet(ni, nout, kss=[9,5,3], conv_sizes=[64, 128, 128], stride=1): 
    "Basic 11 Layer - 1D resnet builder"
    layers = []
    sizes = zip([ni]+conv_sizes, conv_sizes)
    for n1, n2 in sizes:
            layers += [ConvLayer(n1, n2, ks=kss[0], stride=stride, ndim=1),
                       res_block_1d(n2, kss[1:3])]
    return nn.Sequential(*layers, 
                         AdaptiveConcatPool1d(),
                         Flatten(),
                        *LinBnDrop(2*n2, nout, p=0.1)
                        )

In [ ]:
resnet = create_resnet(1, 37)
resnet

Sequential(
  (0): ConvLayer(
    (0): Conv1d(1, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): SequentialEx(
    (layers): ModuleList(
      (0): ConvLayer(
        (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): ConvLayer(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): MergeLayer()
    )
  )
  (2): ConvLayer(
    (0): Conv1d(64, 128, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): SequentialEx(
    (layers): ModuleList(
      (

In [ ]:
resnet(ts).shape

torch.Size([2, 37])

## Inception Time
> InceptionTime: Finding AlexNet for Time SeriesClassification
The original paper repo is [here](https://github.com/hfawaz/InceptionTime)

This module is to be used within a `SequentialEx` block, it acces `orig` attribute to create the shortcut.

In [ ]:
#export
class Shortcut(Module):
    "Merge a shortcut with the result of the module by adding them. Adds Conv, BN and ReLU"
    def __init__(self, ni, nf, act_fn=act_fn): 
        self.act_fn=act_fn
        self.conv=ConvLayer(ni, nf, ks=1, ndim=1)
        self.bn=nn.BatchNorm1d(nf)
    def forward(self, x): return act_fn(x + self.bn(self.conv(x.orig)))

The actual inception module as described on the paper.

In [ ]:
#export
def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv1d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

In [ ]:
#export
class InceptionModule(Module):
    "The inception Module from `ni` inputs to len('kss')*`nb_filters`+`bottleneck_size`"
    def __init__(self, ni, nb_filters=32, kss=[39, 19, 9], bottleneck_size=32, stride=1):
        if (bottleneck_size>0 and ni>1): self.bottleneck = conv(ni, bottleneck_size, 1, stride)
        else: self.bottleneck = noop
        self.convs = nn.ModuleList([conv(bottleneck_size if (bottleneck_size>1 and ni>1) else ni, nb_filters, ks) for ks in kss])
        self.conv_bottle = nn.Sequential(nn.MaxPool1d(3, stride, padding=1), conv(ni, nb_filters, 1))
        self.bn_relu = nn.Sequential(nn.BatchNorm1d((len(kss)+1)*nb_filters), nn.ReLU())
    def forward(self, x):
        bottled = self.bottleneck(x)
        return self.bn_relu(torch.cat([c(bottled) for c in self.convs]+[self.conv_bottle(x)], dim=1))

`InceptionModule(64. nb_filters=32)`: will create a 64 channel input module to 3 x 32 channel kernels stacked together with a 32 bottleneck, to form a 128 channel output.

In [ ]:
im = InceptionModule(8, nb_filters=32)
im

InceptionModule(
  (bottleneck): Conv1d(8, 32, kernel_size=(1,), stride=(1,), bias=False)
  (convs): ModuleList(
    (0): Conv1d(32, 32, kernel_size=(39,), stride=(1,), padding=(19,), bias=False)
    (1): Conv1d(32, 32, kernel_size=(19,), stride=(1,), padding=(9,), bias=False)
    (2): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
  )
  (conv_bottle): Sequential(
    (0): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (1): Conv1d(8, 32, kernel_size=(1,), stride=(1,), bias=False)
  )
  (bn_relu): Sequential(
    (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU()
  )
)

In [ ]:
im(torch.rand(2, 8, 100)).shape

torch.Size([2, 128, 100])

### Creating InceptionTime:
- ni: number of input channels
- nout: number of outputs, should be equal to the number of classes for classification tasks.
- kss: kernel sizes for the inception Block.
- bottleneck_size: The number of channels on the convolution bottleneck.
- nb_filters: Channels on the convolution of each kernel.
- head: True if we want a head attached.

In [ ]:
#export
def create_inception(ni, nout, kss=[39, 19, 9], depth=6, bottleneck_size=32, nb_filters=32, head=True):
    "Creates an InceptionTime arch from `ni` channels to `nout` outputs."
    layers = []
    n_ks = len(kss) + 1
    for d in range(depth):
        im = SequentialEx(InceptionModule(ni if d==0 else n_ks*nb_filters, kss=kss, bottleneck_size=bottleneck_size))
        if d%3==2: im.append(Shortcut(n_ks*nb_filters, n_ks*nb_filters))      
        layers.append(im)
    head = [AdaptiveConcatPool1d(), Flatten(), nn.Linear(2*n_ks*nb_filters, nout)] if head else []
    return  nn.Sequential(*layers, *head)

In [ ]:
inception = create_inception(1, 37)
inception

Sequential(
  (0): SequentialEx(
    (layers): ModuleList(
      (0): InceptionModule(
        (convs): ModuleList(
          (0): Conv1d(1, 32, kernel_size=(39,), stride=(1,), padding=(19,), bias=False)
          (1): Conv1d(1, 32, kernel_size=(19,), stride=(1,), padding=(9,), bias=False)
          (2): Conv1d(1, 32, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
        )
        (conv_bottle): Sequential(
          (0): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
          (1): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
        )
        (bn_relu): Sequential(
          (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
        )
      )
    )
  )
  (1): SequentialEx(
    (layers): ModuleList(
      (0): InceptionModule(
        (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
        (convs): ModuleList(
          (0): Conv1d(32, 32, kernel_size=(3

In [ ]:
inception(ts).shape

torch.Size([2, 37])

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_models.ipynb.
Converted 03_tabular.ipynb.
Converted 04_testing.ipynb.
Converted 99_index.ipynb.
